In [1]:
# Taking some random text from wikipedia

text = """Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. 
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. 
Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. 
Kip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. 
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles. 
Interstellar uses extensive practical and miniature effects, and the company Double Negative created additional digital effects.

Interstellar premiered in Los Angeles on October 26, 2014. In the United States, it was first released on film stock, expanding to venues using digital projectors. The film received generally positive reviews from critics and grossed over $677 million worldwide ($715 million after subsequent re-releases), making it the tenth-highest-grossing film of 2014. 
It has been praised by astronomers for its scientific accuracy and portrayal of theoretical astrophysics.[5][6][7] Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades."""

#### Manual Approach

In [2]:
# Say LLM token limit is 100, in that case we can do simple thing such as this
text[0:100]

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher N'

In [3]:
# Well but we want complete words and want to do this for entire text, may be we can use Python's split funciton
words = text.split(" ")
len(words)

264

In [4]:
chunks = []
s = " "
for word in words:
    s = s + word + " "
    if len(s) > 100:
        chunks.append(s)
        s = ""

print(chunks)


[' Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. ', '\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, ', 'and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and ', 'famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of ', 'a new home for humankind.\n\nBrothers Christopher and Jonathan Nolan wrote the screenplay, which had its ', 'origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. ', '\nKip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive ', 'producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. \nCinematographer ', 'Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal ', 'photogra

In [5]:
chunks[:2]

[' Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. ',
 '\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, ']

Splitting data into chunks can be done in native python but it is a tidious process. Also if necessary, you may need to experiment with various delimiters in an iterative manner to ensure that each chunk does not exceed the token length limit of the respective LLM.

##### Langchain provides a better way through text splitter classes.

### Using Text Splitter Classes from Langchain

#### CharacterTextSplitter

In [6]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 200,
    chunk_overlap = 0
)

In [7]:
chunks = splitter.split_text(text)
len(chunks)

Created a chunk of size 210, which is longer than the specified 200
Created a chunk of size 208, which is longer than the specified 200
Created a chunk of size 358, which is longer than the specified 200


9

In [8]:
for chunk in chunks:
    print(len(chunk))

105
120
210
181
197
207
128
357
253


As you can see, all though we gave 200 as a chunk size since the split was based on \n, it ended up creating chunks that are bigger than size 200.

Another class from Langchain can be used to recursively split the text based on a list of separators. This class is RecursiveTextSplitter. Let's see how it works

### RecursiveTextSplitter

In [9]:
text

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.\n\nBrothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg. \nKip Thorne, a Caltech theoretical physicist and 2017 Nobel laureate in Physics,[4] was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar. \nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm. Principal photography began in late 2013 and took place 

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 200,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of \
                           # string however you can pass any token counter)
)

# 1st whole text is splitted using the "\n\n"
# Then we will get 3 chunks
# If any of the returned chunks are larger than 200 size 
# It will be splitted using the "\n"
# If it is still lager than 200 size
# It will be seperated using " "
# Then it will merge if the size is lesser 

In [11]:
chunks = r_splitter.split_text(text)

for chunk in chunks:
    print(len(chunk))

105
120
199
10
181
197
198
8
128
191
165
198
54


In [12]:
first_split = text.split("\n\n")[0]
first_split

'Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. \nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. \nSet in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.'

In [13]:
len(first_split)

439

In [14]:
# Recursive text splitter uses a list of separators, i.e. separators = ["\n\n", "\n", "."]
# So now it will first split using \n\n and then if the resulting chunk size is greater than 
# the chunk_size parameter which is 200 in our case, then it will use the next separator which is \n

second_split = first_split.split("\n")
second_split

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan. ',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine. ',
 'Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.']

In [15]:
for split in second_split:
    print(len(split))

106
121
210


In [16]:
# Third split exceeds chunk size 200. Now it will further try to split that using the third separator which is ' ' (space)
second_split[2]

'Set in a dystopian future where humanity is embroiled in a catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for humankind.'

In [ ]:
# When you split this using space (i.e. second_split[2].split(" ")), it will separate out each word and 
# then it will merge those chunks such that their size is close to 200